In [55]:
import numpy as np
import torch 
import pyro
import re
import pandas as pd

import importlib
import time
import pickle

from granch_utils import init_model_tensor, main_sim_tensor, lesioned_sim, compute_prob_tensor, init_params_tensor, num_stab_help
#importlib.reload(granch_utils)
importlib.reload(num_stab_help)
importlib.reload(init_model_tensor)
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

<module 'granch_utils.lesioned_sim' from '/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/lesioned_sim.py'>

In [94]:
importlib.reload(main_sim_tensor)
importlib.reload(lesioned_sim)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

BATCH_INFO = {
    "jitter_n": 1, 
    "total_batch_n": 1, 
    "jitter_mode": "sampling"
}

GRID_INFO = {
    "grid_mu_start": -4, "grid_mu_end": 4, "grid_mu_step": 20, 
    "grid_sigma_start": 0.001, "grid_sigma_end": 1.8, "grid_sigma_step": 20, 
    "grid_y_start": -4, "grid_y_end": 4, "grid_y_step": 20, 
    "grid_epsilon_start": 0.001, "grid_epsilon_end": 1.8, "grid_epsilon_step": 20, 
    "hypothetical_obs_grid_n": 10
}


BATCH_GRID_INFO = num_stab_help.get_batch_grid(BATCH_INFO, GRID_INFO)

PRIOR_INFO = {
    "mu_prior": 0,  
    "V_prior": 3, 
    "alpha_prior": 1, 
    "beta_prior": 2, 
    "epsilon": 0.001, "mu_epsilon": 0.001, "sd_epsilon": 4, 
    "hypothetical_obs_grid_n": 10, 
    "world_EIGs": 0.0001, "max_observation": 500
}

tensor_stimuli = num_stab_help.sample_spore_experiment(1)

tensor_model =  init_model_tensor.granch_model(PRIOR_INFO['max_observation'], tensor_stimuli[0])

params = init_params_tensor.granch_params(
                grid_mu =  BATCH_GRID_INFO["grid_mus"][0].to(device),
                grid_sigma = BATCH_GRID_INFO["grid_sigmas"][0].to(device),
                grid_y = BATCH_GRID_INFO["grid_ys"][0].to(device),
                grid_epsilon = BATCH_GRID_INFO["grid_epsilons"][0].to(device),
                hypothetical_obs_grid_n = PRIOR_INFO["hypothetical_obs_grid_n"], 
                mu_prior = PRIOR_INFO["mu_prior"],
                V_prior = PRIOR_INFO["V_prior"], 
                alpha_prior = PRIOR_INFO["alpha_prior"], 
                beta_prior = PRIOR_INFO["beta_prior"],
                epsilon  = PRIOR_INFO["epsilon"], 
                mu_epsilon = PRIOR_INFO["mu_epsilon"], 
                sd_epsilon = PRIOR_INFO["sd_epsilon"], 
                world_EIGs = PRIOR_INFO["world_EIGs"],
                max_observation = PRIOR_INFO["max_observation"], 
                forced_exposure_max= np.nan)
        
            # add the various different cached bits
params.add_meshed_grid()
params.add_lp_mu_sigma()
params.add_y_given_mu_sigma()
params.add_lp_epsilon()
params.add_priors()

#res = main_sim_tensor.granch_main_simulation(params, tensor_model, tensor_stimuli[0])
#res = lesioned_sim.granch_no_learning_simulation(params, tensor_model, tensor_stimuli[0])
res = lesioned_sim.granch_no_noise_simulation(params, tensor_model, tensor_stimuli[0])

print(res.behavior)
#main_sim_tensor.granch_main_simulation(PRIOR_INFO, tensor_model, tensor_stimuli)


/Users/caoanjie/Desktop/projects/looking_time_models/02_pyGRANCH/granch_utils/init_model_tensor.py:279: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.199' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.all_observations.loc[self.current_t]  = Normal(current_stimulus, noise_epsilon).sample().tolist()


    stimulus_id       EIG Look_away
0             0  0.486392     False
1             0   0.45973     False
2             0  0.306492     False
3             0  0.168279     False
4             0  0.087707     False
..          ...       ...       ...
495         NaN       NaN       NaN
496         NaN       NaN       NaN
497         NaN       NaN       NaN
498         NaN       NaN       NaN
499         NaN       NaN       NaN

[500 rows x 3 columns]


In [89]:
d = pd.read_pickle("no_learning/09-17-11:31:45.831-spore-adult.pickle")


main_df = d.dropna(subset = ["stimulus_id"])
counts = main_df.groupby(['j_i',  "stimulus_id", "stim_squence", "violation_type"]).agg({'epsilon': 'first', 'b_val': 'first', 'd_val': 'first', 'mu_prior' : 'first',
                                                                                                                'mu_prior': 'first','v_prior': 'first','alpha_prior': 'first','beta_prior': 'first',
      
                                                                                              'epsilon': 'first'}).reset_index()
        

counts["n_sample"] =  main_df.groupby([ 'j_i',  "stimulus_id", "stim_squence", "violation_type"], as_index=False).count()['Look_away']

#d.groupby(["j_i","stimulus_id", "stim_squence", "violation_type"], as_index=False).count()['Look_away']
print(counts)
 

     j_i  stimulus_id stim_squence violation_type  epsilon  b_val  d_val  \
0      0            0       BBBBBB       identity   0.0001  0.116  0.223   
1      0            1       BBBBBB       identity   0.0001  0.116  0.223   
2      0            2       BBBBBB       identity   0.0001  0.116  0.223   
3      0            3       BBBBBB       identity   0.0001  0.116  0.223   
4      0            4       BBBBBB       identity   0.0001  0.116  0.223   
..   ...          ...          ...            ...      ...    ...    ...   
115   19            1       BBBBBB       identity   0.0001  0.116  0.223   
116   19            2       BBBBBB       identity   0.0001  0.116  0.223   
117   19            3       BBBBBB       identity   0.0001  0.116  0.223   
118   19            4       BBBBBB       identity   0.0001  0.116  0.223   
119   19            5       BBBBBB       identity   0.0001  0.116  0.223   

     mu_prior  v_prior  alpha_prior  beta_prior  n_sample  
0           0        1     

In [93]:
import os
import pandas as pd

def count_until_look_away(group):
    return group['Look_away'].tolist().index(True) if True in group['Look_away'].tolist() else len(group)

folder_path = "no_learning/"
df_list = []

# infant or adult runs
paradigm = 'adult'

# spore or unity
stims = 'spore'

# get list of file names
file_names = [f for f in os.listdir(folder_path) if f.endswith('.pickle') and paradigm in f and stims in f]

print("file_names:", file_names[0:20])

for idx, file_name in enumerate(file_names):
        file_path = os.path.join(folder_path, file_name)
        try:
            df = pd.read_pickle(file_path)
            df["batch_id"] = idx
            df_list.append(df)
        except:
             print('error encountered for: ', file_path)
             
        main_df = pd.concat(df_list)
        main_df = main_df.dropna(subset = ["stimulus_id"])
        counts = main_df.groupby(['batch_id', 'j_i',  "stimulus_id", "stim_squence", "violation_type"]).agg({'epsilon': 'first', 'b_val': 'first', 'd_val': 'first', 'mu_prior' : 'first',
                                                                                                                'mu_prior': 'first','v_prior': 'first','alpha_prior': 'first','beta_prior': 'first',
                                                                                                    'epsilon': 'first'}).reset_index()
        
        counts["n_sample"] =  main_df.groupby(['batch_id', 'j_i',  "stimulus_id", "stim_squence", "violation_type"], as_index=False).count()['Look_away']
        #print(counts['n_sample'])
        
        counts.to_csv("no_noise"+ ".csv", mode = 'a', index=False, header=False)
        df_list = []


file_names: ['09-17-11:31:59.294-spore-adult.pickle', '09-17-11:32:11.471-spore-adult.pickle', '09-17-11:32:07.314-spore-adult.pickle', '09-17-11:31:55.013-spore-adult.pickle', '09-17-11:31:57.712-spore-adult.pickle', '09-17-11:31:55.483-spore-adult.pickle', '09-17-11:31:49.310-spore-adult.pickle', '09-17-11:31:53.904-spore-adult.pickle', '09-17-11:31:46.328-spore-adult.pickle', '09-17-11:32:02.088-spore-adult.pickle', '09-17-11:32:04.979-spore-adult.pickle', '09-17-11:31:50.344-spore-adult.pickle', '09-17-11:32:01.602-spore-adult.pickle', '09-17-11:31:52.155-spore-adult.pickle', '09-17-11:32:04.506-spore-adult.pickle', '09-17-11:31:52.962-spore-adult.pickle', '09-17-11:32:09.242-spore-adult.pickle', '09-17-11:32:09.080-spore-adult.pickle', '09-17-11:32:10.192-spore-adult.pickle', '09-17-11:32:05.292-spore-adult.pickle']
